In [1]:
import pandas as pd
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from custom_score.utils import cleanString
from datasets_loaders.loaders import load_billsum
from sklearn.cluster import SpectralClustering, Birch
from hdbscan import HDBSCAN

import matplotlib.pyplot as plt
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat

### Dataset

In [ ]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

In [ ]:
subset = dataset.iloc[3:5, :]
subset

### ILP Tuning

In [5]:
sources = [
    "My left knee is higly damaged. I might not be able to run anymore. I am not happy about this news.",
    "I work as a data scientist in an IT company. I love data science. I word there since 2 years.",
    "My neighbord often plays guitar. My mother is a very nice person. Sometimes I accompany him on the piano.",
    "Most days, I pratice sports. I run and go to the gym 5 times a week. It is complementary with intellectual activities."
]

golds = [
    "My articulation hurts.",
    "I am a pasionnate engineer.",
    "Music helps me connect with people.",
    "Physical activities occupy a consequent part of my life."
]

In [6]:
ms = MARSCore(sources, golds, precision_level="c", ratio=2, clusterizer=Birch(), printRange=(range(3)))
ms.compute()

In [7]:
ms.summaries

['',
 'I love data science.',
 'My neighbord often plays guitar.',
 'It is complementary with intellectual activities.']

In [5]:
ms.clusters_labels

[array([0, 0, 0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 1, 2]),
 array([0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 2]),
 array([1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2]),
 array([0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 2])]

In [6]:
_=ms.assess()

KeyboardInterrupt: 

In [ ]:
print(ms)

In [ ]:
ms.visualize(dim=2)

In [ ]:
print(ms.cluster_distribution())